# 02. Grouping and Aggregating with Multiple Columns

### Objectives

+ Use multiple grouping columns
+ Aggregate multiple columns
+ Use multiple aggregating functions
+ Know different syntax for performing an aggregation

### Overview
In this notebook we will learn how to form groups using more than 1 column. We will also aggregate more than one column as well as learn how to apply multiple aggregation functions to each group.

## Adding Years of Experience to City of Houston Data
Before we get started with grouping and aggregating multiple columns, let's read in the City of Houston employee dataset and add the years of experience.

In [1]:
import pandas as pd
emp = pd.read_csv('../data/employee.csv', parse_dates=['hire_date', 'job_date'])
emp.head()

,title,dept,salary,race,gender,hire_date,job_date
0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic,Female,2006-06-12,2012-10-13
1,LIBRARY ASSISTANT,Library,26125.0,Hispanic,Female,2000-07-19,2010-09-18
2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Male,2015-02-03,2015-02-03
3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Male,1982-02-08,1991-05-25
4,ELECTRICIAN,General Services Department,56347.0,White,Male,1989-06-19,1994-10-22


### Calculate years of experience from hire date
The data was pulled on December 1, 2016. Let's use the **`dt`** accessor with the **`year`** attribute to get the year that each employee was hired. We can subtract this year from 2016 to approximate the years of experience and assign it as a new column.

In [2]:
emp['experience'] = 2016 - emp['hire_date'].dt.year

### Take a peak at distribution of experience
Use the **`value_counts`** to get a quick understanding of how experience is distributed.

In [3]:
emp['experience'].value_counts(normalize=True).head(10)

1     0.0735
2     0.0585
9     0.0545
3     0.0510
8     0.0495
10    0.0445
0     0.0420
4     0.0370
7     0.0370
23    0.0370
Name: experience, dtype: float64

In [4]:
emp.head()

,title,dept,salary,race,gender,hire_date,job_date,experience
0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic,Female,2006-06-12,2012-10-13,10
1,LIBRARY ASSISTANT,Library,26125.0,Hispanic,Female,2000-07-19,2010-09-18,16
2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Male,2015-02-03,2015-02-03,1
3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Male,1982-02-08,1991-05-25,34
4,ELECTRICIAN,General Services Department,56347.0,White,Male,1989-06-19,1994-10-22,27


## Review grouping and aggregating with a single column
In the previous notebook, we had a single grouping column, aggregating column, and aggregating function. The following syntax was used as a guide:

**```
df.groupby('<grouping column>').agg({'<aggregating column>':'<aggregating function>'})
```**

Let's see this again by calculating the average years of experience for each gender.

In [5]:
emp.groupby('gender').agg({'experience': 'mean'})

,experience
gender,
Female,12.230514
Male,14.283465


# Grouping with Multiple Columns
To create groups based on distinct groups from multiple columns, we will need to pass a list of these columns to the **`groupby`** method. Let's find the average years of experience for every unique combination of race and gender.

In [6]:
emp.groupby(['race', 'gender']).agg({'experience': 'mean'})

experience
race            gender            
Asian           Female   14.033333
                Male     13.090909
Black           Female   13.249180
                Male     12.513924
Hispanic        Female   10.633094
                Male     12.398827
Native American Female   12.166667
                Male     13.200000
Other           Female    1.000000
                Male      4.000000
White           Female   12.583333
                Male     17.420108

### What happened to our index?
Both race and gender are not columns and have been pushed into the index. This is a special called a **multi-level index** and technically a **`MultiIndex`** object. **`Race`** and **`Gender`** are considered **levels** of the index. They are NOT columns. You'll notice that duplicated values do not repeat in an index when they immediately follow one another.

### The MultiIndex is confusing and not necessary for beginners
In my opinion, this multi-level index only adds to confusion. By default, all grouping columns will be added to the index. From this point on, we will chain the **`reset_index`** method to keep these values as columns.

In [7]:
emp.groupby(['race', 'gender']).agg({'experience': 'mean'}).reset_index()

,race,gender,experience
0,Asian,Female,14.033333
1,Asian,Male,13.090909
2,Black,Female,13.249180
3,Black,Male,12.513924
4,Hispanic,Female,10.633094
5,Hispanic,Male,12.398827
6,Native American,Female,12.166667
7,Native American,Male,13.200000
8,Other,Female,1.000000
9,Other,Male,4.000000


### Isn't it easier to read with a MultiIndex?
The MultiIndex can make the results easier to read, but it makes further data analysis more difficult as you need to become familiar with special syntax just for the MultiIndex. This added complexity for beginners is not worth any benefit.

# Aggregating Multiple Columns
To aggregate multiple columns, add the column name to the dictionary paired with its aggregation function. The aggregation functions can be different.

The following finds the average salary and max years of experience for each gender.

In [3]:
emp.groupby('gender').agg({'salary': 'mean', 
                           'experience': 'max'}).reset_index()

,gender,salary,experience
0,Female,52168.339688,41
1,Male,57354.611917,58


# Grouping and Aggregating with Multiple Columns
We can combine the last two approaches to group with multiple columns along with aggregating multiple columns.

The following finds the mean salary and max experience for every unique combination of race and gender. It might make things more readable by placing the each aggregating column on a separate line.

In [9]:
emp.groupby(['race', 'gender']).agg({'salary': 'mean', 
                                     'experience': 'max'}).reset_index()

,race,gender,salary,experience
0,Asian,Female,63226.300000,35
1,Asian,Male,61033.906667,39
2,Black,Female,48915.421233,37
3,Black,Male,51082.074074,48
4,Hispanic,Female,46503.316176,37
5,Hispanic,Male,54782.819018,41
6,Native American,Female,60238.800000,21
7,Native American,Male,60305.400000,25
8,Other,Female,63785.000000,1
9,Other,Male,38771.000000,4


# Multiple Aggregation Functions
Let's say we want to find the min, max, mean, and median salary for each race. We do this by using a list of aggregating functions as the key in our **`agg`** dictionary.

In [5]:
emp.groupby('race').agg({'salary': ['min', 'max', 'mean', 'median']}) # in dic, use a list 
# multi-level column index

salary                                 
                     min       max          mean   median
race                                                     
Asian            26125.0  163228.0  61660.304762  55461.0
Black            24960.0  275000.0  50137.801493  45744.5
Hispanic         26104.0  165216.0  52345.562771  50378.0
Native American  26125.0   98536.0  60272.100000  59601.0
Other            38771.0   63785.0  51278.000000  51278.0
White            26125.0  210588.0  64419.799012  62540.0

## What's up with those column names???
The column names probably look pretty bizarre to you. Although it doesn't take much effort to decipher what each column means, the column names are not particularly friendly to work with.

Pandas created a **multi-level column index** with two levels. These are difficult to work with. There isn't a standard way to deal with them like we did with the multi-level index from above.

## Renaming all the columns
I recommend renaming all the columns after the aggregation. This is quite simple, but tedious. Simply assign the DataFrame's **`columns`** attribute to a list of desired column names. The list must be the same length as the original.

In [6]:
race_salary = emp.groupby('race').agg({'salary': ['min', 'max', 'mean', 'median']}).reset_index()
race_salary.columns = ['race', 'min salary', 'max salary', 'mean salary', 'median salary']
race_salary

,race,min salary,max salary,mean salary,median salary
0,Asian,26125.0,163228.0,61660.304762,55461.0
1,Black,24960.0,275000.0,50137.801493,45744.5
2,Hispanic,26104.0,165216.0,52345.562771,50378.0
3,Native American,26125.0,98536.0,60272.100000,59601.0
4,Other,38771.0,63785.0,51278.000000,51278.0
5,White,26125.0,210588.0,64419.799012,62540.0


If you are not planning on using the returned DataFrame then you don't need to bother renaming the columns, but having a single level index is going to be much easier to work with than a MultiIndex when you are first beginning your Pandas journey. 

## No added functionality of a MultiIndex
I actually don't think the MultiIndex offers much benefit. All data analysis is possible without it. There are some cool tricks you can do with it, but overall it will not prevent you from achieving any kind of analysis if you do not use it.

# Multiple Grouping Columns, Aggregating Columns, and Aggregating Functions
You can make complex aggregations by having multiple grouping columns, aggregating columns, and aggregating functions.

In [7]:
rg_sal_exp = emp.groupby(['race', 'gender']) \
                .agg({'salary': ['min', 'max', 'mean', 'median'],
                      'experience': ['max', 'std']}).reset_index()
rg_sal_exp

race  gender   salary                                   \
                                 min       max          mean   median   
0             Asian  Female  26125.0  130416.0  63226.300000  57227.5   
1             Asian    Male  27914.0  163228.0  61033.906667  55461.0   
2             Black  Female  24960.0  150416.0  48915.421233  44491.0   
3             Black    Male  26125.0  275000.0  51082.074074  46486.5   
4          Hispanic  Female  26125.0  126115.0  46503.316176  43087.0   
5          Hispanic    Male  26104.0  165216.0  54782.819018  54090.5   
6   Native American  Female  26125.0   98536.0  60238.800000  58855.0   
7   Native American    Male  26125.0   81239.0  60305.400000  60347.0   
8             Other  Female  63785.0   63785.0  63785.000000  63785.0   
9             Other    Male  38771.0   38771.0  38771.000000  38771.0   
10            White  Female  27955.0  178331.0  66793.352941  62264.5   
11            White    Male  26125.0  210588.0  63940.388119  62540.0   

   experience             
          max        std  
0          35   9.535716  
1          39   9.506482  
2          37   9.458890  
3          48  10.224718  
4          37   8.771779  
5          41   9.605721  
6          21   6.675827  
7          25   9.038805  
8           1        NaN  
9           4        NaN  
10         41  10.159591  
11         58  11.689454

Again, I suggest renaming the columns for easier data manipulation.

In [8]:
rg_sal_exp.columns = ['race', 'gender', 'min salary', 'max salary', 'mean salary',
                      'median salary', 'max exp', 'std exp']
rg_sal_exp

,race,gender,min salary,max salary,mean salary,median salary,max exp,std exp
0,Asian,Female,26125.0,130416.0,63226.300000,57227.5,35,9.535716
1,Asian,Male,27914.0,163228.0,61033.906667,55461.0,39,9.506482
2,Black,Female,24960.0,150416.0,48915.421233,44491.0,37,9.458890
3,Black,Male,26125.0,275000.0,51082.074074,46486.5,48,10.224718
4,Hispanic,Female,26125.0,126115.0,46503.316176,43087.0,37,8.771779
5,Hispanic,Male,26104.0,165216.0,54782.819018,54090.5,41,9.605721
6,Native American,Female,26125.0,98536.0,60238.800000,58855.0,21,6.675827
7,Native American,Male,26125.0,81239.0,60305.400000,60347.0,25,9.038805
8,Other,Female,63785.0,63785.0,63785.000000,63785.0,1,NaN
9,Other,Male,38771.0,38771.0,38771.000000,38771.0,4,NaN


# Getting the size of each group
Let's say we just want to know the number of rows in each group. The correct aggregation function is **`size`** and not **`count`** (this returns the number of non-missing values).

In [14]:
emp.groupby(['race', 'gender']).agg({'salary': 'size'}).reset_index()

,race,gender,salary
0,Asian,Female,30
1,Asian,Male,77
2,Black,Female,305
3,Black,Male,395
4,Hispanic,Female,139
5,Hispanic,Male,341
6,Native American,Female,6
7,Native American,Male,5
8,Other,Female,1
9,Other,Male,1


### The aggregating column doesn't matter
The same result will be returned regardless of what aggregating column we use since the size only depends on the number of rows and not on the actual values in the column. Using the department column does not change the output.

In [15]:
emp.groupby(['race', 'gender']).agg({'dept': 'size'}).reset_index()

,race,gender,dept
0,Asian,Female,30
1,Asian,Male,77
2,Black,Female,305
3,Black,Male,395
4,Hispanic,Female,139
5,Hispanic,Male,341
6,Native American,Female,6
7,Native American,Male,5
8,Other,Female,1
9,Other,Male,1


## Alternative Syntax for size
You can call the **`size`** method directly after grouping. This will return the same data as a Series.

In [16]:
emp.groupby(['race', 'gender']).size().reset_index()

,race,gender,0
0,Asian,Female,30
1,Asian,Male,77
2,Black,Female,305
3,Black,Male,395
4,Hispanic,Female,139
5,Hispanic,Male,341
6,Native American,Female,6
7,Native American,Male,5
8,Other,Female,1
9,Other,Male,1


# Exercises

### Problem 1
<span  style="color:green; font-size:16px">For each department and gender find the number of unique position titles, the total number of employees and the average salary. Make sure there is no multi-index for the index or columns.</span>

In [9]:
emp.head(2)

,title,dept,salary,race,gender,hire_date,job_date,experience
0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic,Female,2006-06-12,2012-10-13,10
1,LIBRARY ASSISTANT,Library,26125.0,Hispanic,Female,2000-07-19,2010-09-18,16


In [ ]:
rg_sal_exp = emp.groupby(['race', 'gender']) \
                .agg({'salary': ['min', 'max', 'mean', 'median'],
                      'experience': ['max', 'std']}).reset_index()
rg_sal_exp.columns = ['race', 'gender', 'min salary', 'max salary', 'mean salary',
                      'median salary', 'max exp', 'std exp']

In [27]:
# your code here
p1 = emp.groupby(['dept', 'gender']).agg({'title':['nunique', 'size'], 'salary':'mean'}).reset_index()
p1.head()

dept  gender   title             salary
                                      nunique size          mean
0  Admn. & Regulatory Affairs  Female      16   22  48758.181818
1  Admn. & Regulatory Affairs    Male       7    7  57592.285714
2    City Controller's Office  Female       2    4  58979.500000
3    City Controller's Office    Male       1    1  42640.000000
4                City Council  Female       5    7  59260.000000

In [29]:
p1.columns = ['dept', 'gender','unique count', '#people', 'mean salary']
p1.head()

,dept,gender,unique count,#people,mean salary
0,Admn. & Regulatory Affairs,Female,16,22,48758.181818
1,Admn. & Regulatory Affairs,Male,7,7,57592.285714
2,City Controller's Office,Female,2,4,58979.500000
3,City Controller's Office,Male,1,1,42640.000000
4,City Council,Female,5,7,59260.000000


### Problem 2
<span  style="color:green; font-size:16px">For each department, race and gender find the maximum years of experience and salary.</span>

In [34]:
# your code here
p2 = emp.groupby(['dept', 'race']).agg({'experience':'max', 'salary':'max'}).reset_index()
p2.head()

,dept,race,experience,salary
0,Admn. & Regulatory Affairs,Asian,15,130416.0
1,Admn. & Regulatory Affairs,Black,23,72741.0
2,Admn. & Regulatory Affairs,Hispanic,12,47341.0
3,Admn. & Regulatory Affairs,White,21,140416.0
4,City Controller's Office,Asian,3,59077.0


## Use the college dataset for the rest of the problems

In [35]:
college = pd.read_csv('../data/college.csv')
college.head()

,instnm,city,stabbr,hbcu,menonly,womenonly,relaffil,satvrmid,satmtmid,distanceonly,...,ugds_2mor,ugds_nra,ugds_unkn,pptug_ef,curroper,pctpell,pctfloan,ug25abv,md_earn_wne_p10,grad_debt_mdn_supp
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
3,University of Alabama in Huntsville,Huntsville,AL,0.0,0.0,0.0,0,595.0,590.0,0.0,...,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500,24097
4,Alabama State University,Montgomery,AL,1.0,0.0,0.0,0,425.0,430.0,0.0,...,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600,33118.5


### Problem 3
<span  style="color:green; font-size:16px">Which city name appears the most frequently. Do this in two different ways. Do it once with and once without the `groupby` method?</span>

In [38]:
# your code here
p3 = college.groupby('city').agg({'instnm':'size'}).reset_index()
p3.head()

,city,instnm
0,ARTESIA,1
1,Aberdeen,3
2,Abilene,5
3,Abingdon,2
4,Abington,1


In [42]:
city = college['city']

In [47]:
city.value_counts(normalize=False, sort=True, ascending=False).head(5)

New York       87
Chicago        78
Houston        72
Los Angeles    56
Miami          51
Name: city, dtype: int64

### Problem 4
<span  style="color:green; font-size:16px">Does the city **`Houston`** only appear in the state of **`Texas`**?</span>

In [51]:
# your code here
filt = college['city'] == 'Houston'
filt.head()

0    False
1    False
2    False
3    False
4    False
Name: city, dtype: bool

In [53]:
hou_coll = college[filt]

In [54]:
hou_coll.groupby(['city','stabbr']).agg({'instnm':'size'})

instnm
city    stabbr        
Houston MO           1
        TX          71

### Problem 5
<span  style="color:green; font-size:16px">Find the maximum undergraduate population for each state?</span>

In [60]:
# your code here
college.groupby('stabbr').agg({'ugds':'max'}).head()

,ugds
stabbr,
AK,12865.0
AL,29851.0
AR,21405.0
AS,1276.0
AZ,151558.0


### Problem 6
<span  style="color:green; font-size:16px">Among colleges that have the largest undergrad population for each state, what is the difference between the most and least populous college?</span>

In [56]:
college.head(1)

,instnm,city,stabbr,hbcu,menonly,womenonly,relaffil,satvrmid,satmtmid,distanceonly,...,ugds_2mor,ugds_nra,ugds_unkn,pptug_ef,curroper,pctpell,pctfloan,ug25abv,md_earn_wne_p10,grad_debt_mdn_supp
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,...,0.0,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888


In [68]:
# your code here
p5 = college.groupby('stabbr').agg({'ugds':'max'})
p5.head()

,ugds
stabbr,
AK,12865.0
AL,29851.0
AR,21405.0
AS,1276.0
AZ,151558.0


In [70]:
p5.max() - p5.min()

ugds    150956.0
dtype: float64

### Problem 7: Advanced
<span  style="color:green; font-size:16px">Find the name and population of the largest college per state.</span>

In [72]:
# your code here
college.groupby('stabbr').agg({'instnm':'max', 'ugds':'max'}).head()

,instnm,ugds
stabbr,,
AK,University of Alaska Southeast,12865.0
AL,Virginia College-Montgomery,29851.0
AR,Williams Baptist College,21405.0
AS,American Samoa Community College,1276.0
AZ,Yavapai College,151558.0


### Problem 8
<span  style="color:green; font-size:16px">Do distance only schools tend to have more or less student population than non-distance-only schools?</span>

In [25]:
# your code here


### Problem 9
<span  style="color:green; font-size:16px">Do distance only schools tend to be more or less religously affiliated than non-distance-only schools?</span>

In [26]:
# your code here

### Problem 10
<span  style="color:green; font-size:16px">What state has the lowest percentage of currently operating schools of those that have religious affiliation?</span>

In [27]:
# your code here

### Problem 11
<span  style="color:green; font-size:16px">Trim the **`college`** DataFrame to only the 'race' columns - those beginning with **`ugds_`**. Create a new column called **`ugds_other`** that is the sum of any race column that averages under 4% for the entire dataset.</span>

In [28]:
# your code here

### Problem 12
<span  style="color:green; font-size:16px">Which are top 5 historically black colleges that have the highest white percentage?</span>

In [29]:
# your code here